In [9]:
from model.pretrain import VALOR
import os 
import json
import torch
import torch.nn.functional as F
from torchvision.transforms.transforms import *
from torchvision import transforms
from easydict import EasyDict as edict
import ipdb 
from PIL import Image
import torchaudio
from test_valor import get_model_attr
# import argparse
from model.bert_tokenizer import BertTokenizer




# parser = argparse.ArgumentParser()
# parser.add_argument("--video_path", default=None, type=str)
# parser.add_argument("--audio_path", default=None, type=str)
# parser.add_argument("--task", default=None, type=str)
# parser.add_argument("--question", default=None, type=str)
# parser.add_argument("--model_dir", default=None, type=str)
# parser.add_argument("--config", default=None, type=str)



args = {
    "task": "cap%tva",
    "model_dir": "./output/VALOR_base_msr_cap.pt",
    "config": "./output/VALOR_base/log/hps.json"
}

args = type('allMyFields', (object,), args)


def clean(self, text):
        """remove duplicate spaces, lower and remove punctuations """
        text = ' '.join([i for i in text.split(' ') if i != ''])
        text = text.lower()
        for i in self.punctuations:
            text = text.replace(i,'')
        return text

def get_padded_tokens(self,txt_tokens, type, max_len=None):
    
    max_len = self.max_len if  max_len is None else max_len
    txt_tokens = txt_tokens[:max_len]

    if type=='bert':
        txt_tokens = [self.cls_token] + txt_tokens + [self.sep_token]  
    elif type=='clip':
        txt_tokens = [self.sot_token] + txt_tokens + [self.eot_token] 

    txt_tokens = torch.tensor(txt_tokens, dtype=torch.long)

    output = torch.zeros(max_len + 2, dtype=torch.long)
    output[:len(txt_tokens)] = txt_tokens
    return output


def split(frame_name_lists, sample_num):
    if len(frame_name_lists) < sample_num:   ###padding with the last frame
        frame_name_lists += [frame_name_lists[-1]]*(sample_num - len(frame_name_lists))
    k, m = divmod(len(frame_name_lists), sample_num)
    return [frame_name_lists[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in list(range(sample_num))]



def load_from_pretrained_dir(pretrain_dir):
    checkpoint_dir = os.path.os.path.join(pretrain_dir,'ckpt')
   
    checkpoint_ls = [ i for i in os.listdir(checkpoint_dir) if i.startswith('model_step')]
    checkpoint_ls = [int(i.split('_')[2].split('.')[0]) for i in checkpoint_ls]
    checkpoint_ls.sort()    
    step = checkpoint_ls[-1]
    
    checkpoint_name = 'model_step_'+str(step)+'.pt'
    ckpt_file = os.path.os.path.join(checkpoint_dir, checkpoint_name)
    checkpoint = torch.load(ckpt_file, map_location = 'cpu')
    checkpoint = {k.replace('module.',''):v for k,v in checkpoint.items()}


    pretrain_cfg = edict(json.load(open(os.path.join(pretrain_dir,'log','hps.json'))))
    ### cover model_cfg 
    # cover_cfg=["audio_melbins", "audio_patch_size", "audio_mean", "audio_std",
    #         "audio_frame_shift", "audio_target_length", "video_encoder_type", 
    #         "txt_encoder_type", "multimodal_encoder_type", "audio_encoder_type","caption_type",
    #         "share_txt_and_multimodal","contra_type","multimodal_use_cross_attn", 
    #        "fineweight_type","has_vafusion_encoder",
    #         "late_fusion","cross_attn_type","task_pormpt_as_text","use_task_prompt"]
    # for k in cover_cfg:
    #     if k in pretrain_cfg:
    #         setattr(opts,k,pretrain_cfg[k])

    # opts = 
   

    if  'video_frame_embedding' in checkpoint:
        checkpoint['video_frame_embedding'][:,pretrain_cfg.video_sample_num:] = checkpoint['video_frame_embedding'][:,pretrain_cfg.video_sample_num-1].clone()
    if  'audio_frame_embedding' in checkpoint: 
        checkpoint['audio_frame_embedding'][:,pretrain_cfg.audio_sample_num:] = checkpoint['audio_frame_embedding'][:,pretrain_cfg.audio_sample_num-1].clone()

  
    return checkpoint, pretrain_cfg




#pretrain_dir = '/public/chensihan/projects/VALOR/output/VALOR_base/caption-msrvtt-lr9e-6-bs64-epoch5-test10frame-0.05warmup-train6frame'


# checkpoint,pretrain_cfg = load_from_pretrained_dir(args.model_dir)
ckpt_file = args.model_dir
checkpoint = torch.load(ckpt_file, map_location = 'cpu')
checkpoint = {k.replace('module.',''):v for k,v in checkpoint.items()}

pretrain_cfg = edict(json.load(open(args.config)))

if  'video_frame_embedding' in checkpoint:
    checkpoint['video_frame_embedding'][:,pretrain_cfg.video_sample_num:] = checkpoint['video_frame_embedding'][:,pretrain_cfg.video_sample_num-1].clone()
if  'audio_frame_embedding' in checkpoint: 
    checkpoint['audio_frame_embedding'][:,pretrain_cfg.audio_sample_num:] = checkpoint['audio_frame_embedding'][:,pretrain_cfg.audio_sample_num-1].clone()

from model.pretrain import VALOR
model = VALOR.from_pretrained(pretrain_cfg,checkpoint)

model.eval().cuda()


def process_video(video_path, args):
    
    
    #video_path = '/public/chensihan/datasets/music-avqa/all_videos/00000002.mp4'
    # video_path = args.video_path
    video_name = video_path.split('/')[-1].split('.')[0]
    output_dir = f'./inference/{video_name}'
    fps_frame_dir = os.path.join(output_dir, f"frames_fps1")
    os.makedirs(fps_frame_dir, exist_ok=True)
    cmd = "ffmpeg -loglevel error -i {} -vsync 0 -f image2 -vf fps=fps={:.02f} -qscale:v 2 {}/frame_%04d.jpg".format(
            video_path, 1, fps_frame_dir)
    os.system(cmd)
    # Extract Audio
    audio_file_path = os.path.join(output_dir,video_name+'.mp4')
    
    
    cmd = "ffmpeg -i {} -loglevel error -f wav -vn -ac 1 -ab 16k -ar {} -y {}".format(
            video_path, 22050, audio_file_path)
    os.system(cmd)
    
    
    
    
    
    if pretrain_cfg.video_encoder_type.startswith('clip'):
        mean = [0.48145466, 0.4578275, 0.40821073] 
        std  = [0.26862954, 0.26130258, 0.27577711]
    else:       
        mean = [0.485, 0.456, 0.406]
        std  = [0.229, 0.224, 0.225]
    
    
    
    test_transforms = transforms.Compose([Resize((224,224)),
                                                Normalize(mean,std)])
    
    
    
    
    
    video_pixels = []        
    frame_path = fps_frame_dir
    frames = os.listdir(frame_path)
    frames.sort()   ### ['img_0001.jpg','img_0002.jpg',...]
    sample_num = 8
    frames_splited = split(frames,sample_num)    
    
    sample_idx = [i[(len(i)+1)//2-1] for i in frames_splited]
    for i in range(sample_num):
        frame = Image.open(os.path.join(frame_path,sample_idx[i]))
        frame = transforms.ToTensor()(frame)   ## frame: 3XhXw
        video_pixels.append(frame.unsqueeze(0))
    video_pixels = torch.cat(video_pixels,dim=0)   ### nX3xHxW
    
    video_pixels = test_transforms(video_pixels)     
    
    
     
        
    
    
    
      
            
            
    
    wav_file = audio_file_path
    if not os.path.exists(wav_file):
        audio = torch.zeros(self.sample_num, self.melbins, self.target_length)
    
    
    else:
    
        target_length = 512
        sample_num=1
        waveform, sr = torchaudio.load(wav_file)
    
        waveform = waveform - waveform.mean()
        fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                window_type='hanning', num_mel_bins=pretrain_cfg.audio_melbins, dither=0.0, frame_shift=pretrain_cfg.audio_frame_shift)
    
                                    #### fbank shape :(src_length,64)
        src_length = fbank.shape[0]
    
        # #### sample 
        output_slices = []
    
        pad_len = target_length - src_length % target_length
        fbank = torch.nn.ZeroPad2d((0, 0, 0, pad_len))(fbank)
        total_slice_num = fbank.shape[0] // target_length
        total_slice_num = list(range(total_slice_num))
        total_slice_num = split(total_slice_num, sample_num)
    
    
        sample_idx = [i[(len(i)+1)//2-1] for i in total_slice_num]
    
    
        for i in sample_idx:
            output_slices.append(fbank[i*target_length : (i+1)*target_length])
    
        fbank = torch.stack(output_slices,dim=0).permute(0,2,1)   
    
            
    
        ### normalization
        fbank = (fbank - pretrain_cfg.audio_mean) / (pretrain_cfg.audio_std * 2)
    
        #return fbank.permute(1,0)  ### 128, target_length
     
               
    
    print(video_pixels.shape)
    print(fbank.shape)
    
    
    
    if args.task.startswith('cap'):
        if args.task == 'cap%tva':
    
    
            batch =   {'ids': None,
                        'txt_tokens': None,
                        'video_pixels': video_pixels.unsqueeze(0).cuda(),
                        'audio_spectrograms': fbank.unsqueeze(0).cuda(),
                        'ids_txt':None,
                        'sample_num':None}
                        
            evaluation_dict = model(batch, 'cap%tva', compute_loss=False)
    
            # print(evaluation_dict)
    
    
            sents = evaluation_dict['generated_sequences_t_va']
            sents = get_model_attr(model, 'decode_sequence')(sents.data)

            return sents
            print(sents)
        
        elif args.task == 'cap%tv':
    
            batch =   {'ids': None,
                        'txt_tokens': None,
                        'video_pixels': video_pixels.unsqueeze(0).cuda(),
                        'ids_txt':None,
                        'sample_num':None}
                        
            evaluation_dict = model(batch, 'cap%tva', compute_loss=False)
    
            # print(evaluation_dict)
    
    
            sents = evaluation_dict['generated_sequences_t_v']
            sents = get_model_attr(model, 'decode_sequence')(sents.data)
    
            print(sents)
        
        else:
            raise NotImplementedError
        
    
    
    elif args.task.startswith('qa'):
        assert args.question is not None
        bert_tokenizer = BertTokenizer("./pretrained_weights/bert-base-uncased-vocab.txt")
        cls_token = bert_tokenizer.convert_tokens_to_ids(['[CLS]'])[0]
        sep_token = bert_tokenizer.convert_tokens_to_ids(['[SEP]'])[0]
        
    
        tokenized_text = bert_tokenizer.tokenize(args.question)
        txt_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
    
        txt_tokens = [cls_token] + txt_tokens + [sep_token]  
    
        txt_tokens = {'bert_tokens':torch.tensor(txt_tokens, dtype=torch.long).unsqueeze(0).cuda()}
    
    
        if args.task == 'qa%tva':
            batch =   {'ids': None,
                        'question_tokens': txt_tokens,
                        'audio_spectrograms': fbank.unsqueeze(0).cuda(),
                        'video_pixels': video_pixels.unsqueeze(0).cuda(),
                        'ids_txt':None,
                        'sample_num':[1]}
    
                            
            evaluation_dict = model(batch, 'qa%tva', compute_loss=False)
    
            # print(evaluation_dict)
    
    
            sents = evaluation_dict['generated_answers_t_va']
            sents = get_model_attr(model, 'decode_sequence')(sents.data)
    
            print(sents)
        
        elif args.task == 'qa%tv':
            batch =   {'ids': None,
                        'question_tokens': txt_tokens,
                        'video_pixels': video_pixels.unsqueeze(0).cuda(),
                        'ids_txt':None,
                        'sample_num':[1]}
    
                            
            evaluation_dict = model(batch, 'qa%tv', compute_loss=False)
    
            # print(evaluation_dict)
    
    
            sents = evaluation_dict['generated_answers_t_v']
            sents = get_model_attr(model, 'decode_sequence')(sents.data)
    
            print(sents)
        


01/08/2025 08:54:11 - INFO - __main__ -   unexpected_keys in video encoder: []
01/08/2025 08:54:11 - INFO - __main__ -   unexpected_keys in audio encoder: ['audio_embeddings.distill_token']
01/08/2025 08:54:12 - INFO - __main__ -   unexpected_keys in multimodal encoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
01/08/2025 08:54:12 - INFO - __main__ -   missing_keys in multimodal encoder: ['embeddings.prompt_embedding.weight', 'encoder.layer.0.cross_attn.cross.query.weight', 'encoder.layer.0.cross_attn.cross.query.bias', 'encoder.layer.0.cross_attn.cross.key.weight', 'encoder.layer.0.cross_attn.cross.key.bias', 'encoder.layer.0.cross_attn.cross.value.weight', 'encoder.layer.0.cross_attn.cross.value.bias', 'encoder.layer.0.cross_attn.output.de

In [10]:
process_video("./sample/lovely_runner_scene_55.mp4", args)

torch.Size([8, 3, 224, 224])
torch.Size([1, 64, 512])


['a man is singing and playing the piano']

In [11]:
import json
sample = json.load(open("./sample.json", "r"))

In [12]:
sample

[{'source_id': 'Cõng anh mà chạy tập 1',
  'scene_id': 0,
  'name': 'name0',
  'start_time': 0.0,
  'end_time': 24.033333333333335,
  'image_captioning': 'the kite is white, poster on the wall, yellow suitcase on top of a white suitcase, the wall is white, white sign with red lettering, the button is white, purple tie on the man, a sign with a phone number, the number is silver, the word bus in purple letters',
  'translated_transcript': '',
  'transcript': '',
  'character': [],
  'score': 7,
  'category': 'romantic',
  'description': {'text': 'The scene features a white kite hanging on a wall next to a yellow suitcase atop a white one. The background is predominantly white. A white button is visible, and a man wearing a purple tie is present. There\'s also a sign with a phone number displayed, and the word "bus" is in purple letters.'}},
 {'source_id': 'Cõng anh mà chạy tập 1',
  'scene_id': 1,
  'name': 'name1',
  'start_time': 24.033333333333335,
  'end_time': 41.6,
  'image_captio

In [14]:
from tqdm import tqdm

for scene in tqdm(sample):
    scene["valor_captioning"] = process_video(f"./sample/lovely_runner_scene_{scene['scene_id']}.mp4", args)


  0%|          | 0/85 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224])
torch.Size([1, 64, 512])



  1%|          | 1/85 [00:01<02:15,  1.61s/it]

torch.Size([8, 3, 224, 224])
torch.Size([1, 64, 512])



  2%|▏         | 2/85 [00:03<02:03,  1.48s/it]

...


 98%|█████████▊| 83/85 [02:08<00:02,  1.49s/it]

torch.Size([8, 3, 224, 224])
torch.Size([1, 64, 512])



 99%|█████████▉| 84/85 [02:09<00:01,  1.51s/it]

torch.Size([8, 3, 224, 224])
torch.Size([1, 64, 512])



100%|██████████| 85/85 [02:10<00:00,  1.54s/it]


In [16]:
sample

[{'source_id': 'Cõng anh mà chạy tập 1',
  'scene_id': 0,
  'name': 'name0',
  'start_time': 0.0,
  'end_time': 24.033333333333335,
  'image_captioning': 'the kite is white, poster on the wall, yellow suitcase on top of a white suitcase, the wall is white, white sign with red lettering, the button is white, purple tie on the man, a sign with a phone number, the number is silver, the word bus in purple letters',
  'translated_transcript': '',
  'transcript': '',
  'character': [],
  'score': 7,
  'category': 'romantic',
  'description': {'text': 'The scene features a white kite hanging on a wall next to a yellow suitcase atop a white one. The background is predominantly white. A white button is visible, and a man wearing a purple tie is present. There\'s also a sign with a phone number displayed, and the word "bus" is in purple letters.'},
  'valor_captioning': ['a girl is holding an umbrella']},
 {'source_id': 'Cõng anh mà chạy tập 1',
  'scene_id': 1,
  'name': 'name1',
  'start_time'